# Regularisation by Kamishima - Recruiting data

This notebook contains the implementation of the in-processing fairness intervention introduced in [Fairness-Aware Classifier with Prejudice Remover Regularizer](https://link.springer.com/chapter/10.1007/978-3-642-33486-3_3) by Kamishima et al. (2012) as part of the IBM AIF360 fairness tool box github.com/IBM/AIF360.

The intervention achieves demographic parity in a logistic regression classifier which is based on maximising the sum between utility expressed via probabilities of classifying data points correctly given their features and further a regularisation term that incorporates the level of unfairness in the classifier.

In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np
import plotly.graph_objs as go
from aif360.datasets import StandardDataset
from aif360.algorithms.inprocessing import PrejudiceRemover
from helpers.fairness_measures import accuracy

In [ ]:
from helpers import export_plot

## Load data

We have committed preprocessed data to the repository for reproducibility and we load it here. Check out the preprocessing notebook for details on how this data was obtained.

In [ ]:
artifacts_dir = Path("../../../artifacts")

In [ ]:
# override data_dir in source notebook
# this is stripped out for the hosted notebooks
artifacts_dir = Path("../../../../artifacts")

In [ ]:
data_dir = artifacts_dir / "data" / "recruiting"

train = pd.read_csv(data_dir / "processed" / "train.csv")
val = pd.read_csv(data_dir / "processed" / "val.csv")
test = pd.read_csv(data_dir / "processed" / "test.csv")

In order to process data for our fairness intervention we need to define special dataset objects which are part of every intervention pipeline within the IBM AIF360 toolbox. These objects contain the original data as well as some useful further information, e.g., which feature is the protected attribute as well as which column corresponds to the label.

In [ ]:
train_sds = StandardDataset(
    train,
    label_name="employed_yes",
    favorable_classes=[1],
    protected_attribute_names=["race_white"],
    privileged_classes=[[1]],
)
test_sds = StandardDataset(
    test,
    label_name="employed_yes",
    favorable_classes=[1],
    protected_attribute_names=["race_white"],
    privileged_classes=[[1]],
)
val_sds = StandardDataset(
    val,
    label_name="employed_yes",
    favorable_classes=[1],
    protected_attribute_names=["race_white"],
    privileged_classes=[[1]],
)

## Demographic parity

Here, we address demographic parity by learning a fair logisitc regression on the trainig data. Subsequently, we apply the learnt model to the test data and analyse the outcomes for fairness and accuracy. The model training allows the specification of a parameter $\eta$, which controls the loss in accuracy for an increase in fairness. The larger eta the higher the obtained fairness on average.

The user is encouraged to consider the influence of the choice of the parameter $\eta$. However, since the learning of the fair model is for some choice of $\eta$ quite slow, we generated the following predictions with $\eta=10$. Since we cannot save the model we saved the predictions generated by the model instead. Since the training is deterministic, the same choice of $\eta$ should lead to same results.

In [ ]:
# PR = PrejudiceRemover(eta=25.0, sensitive_attr="race_white", class_attr="employed_yes")
# PR.fit(train_sds)

### Apply model
On test data.

In [ ]:
# test_sds_pred = PR.predict(test_sds)
# test_scores = test_sds_pred.scores.flatten()

In [ ]:
test_scores = np.load(
    artifacts_dir / "models" / "recruiting" / "kamishima_test_scores.npy"
)

## Analyse fairness and accuracy

In [ ]:
print("Accuracy =", accuracy(test_scores, test.employed_yes))
print(
    "Female accuracy =",
    accuracy(
        test_scores[test.race_white == 0],
        test.employed_yes[test.race_white == 0],
    ),
)
print(
    "Male accuracy =",
    accuracy(
        test_scores[test.race_white == 1],
        test.employed_yes[test.race_white == 1],
    ),
)
print("Mean female score =", test_scores[test.race_white == 0].mean())
print("Mean male score =", test_scores[test.race_white == 1].mean())

In [ ]:
dp_box = go.Figure(
    data=[
        go.Box(
            x=[race] * (test.race_white == race).sum(),
            y=test_scores[test.race_white == race],
            name="White" if race else "Black",
        )
        for race in range(2)
    ]
)
dp_box

In [ ]:
export_plot(dp_box, "kamishima-dp.json")